In [1]:
import csv
import pandas as pd
import numpy as np
import re
import os

from functools import reduce

In [2]:

# set filepath where model outputs are located
output_filepath = r'C:\Users\calisy\OneDrive\Energy Research\NG Project\code\Gas Model\Gas Model 6 - Weekly Storage Sets Price - Outputs\model_outputs'

# create separate folder in outputs to hold review files
output_review_filepath = r'C:\Users\calisy\OneDrive\Energy Research\NG Project\code\Gas Model\Gas Model 6 - Weekly Storage Sets Price - Outputs\output_review_files'

# Read in the data

output_filenames = ['duals.csv',
                    'qstep_production.csv',
                    'ftariff_step_flow.csv',
                    'total_pipeline_flows.csv',
                    'node_storage_withdrawal.csv',
                    'node_storage_injection.csv',
                    'node_production_slack.csv',
                    'pipeline_slack.csv',
                    'node_storage_slack.csv',
                    'daily_node_production.csv',
                    'daily_node_demand.csv'
                    ]

df_lst = []

for filename in output_filenames:
    df = pd.read_csv(os.path.join(output_filepath, filename))
    df_lst.append(df)

# pipeline outputs
df_qstep_production = df_lst[1]
df_ftariff_step_flow = df_lst[2]
df_total_pipeline_flows = df_lst[3]
df_pipeline_slack = df_lst[7]

# node outputs
df_duals = df_lst[0]
df_node_storage_withdrawal = df_lst[4]
df_node_storage_injection = df_lst[5]
df_node_production_slack = df_lst[6]
df_node_storage_slack = df_lst[8]
df_daily_node_production = df_lst[9]
df_daily_node_demand = df_lst[10]


In [3]:
df_duals
df_total_pipeline_flows

,line,day,total_pipeline_flow
0,AL_to_FL,1,3.243596
1,AL_to_GA,1,3.484197
2,AL_to_MS,1,0.000000
3,AL_to_TN,1,0.000000
4,AZ_to_CA,1,3.318472
...,...,...,...
1549,CN_W_to_ND_CN_W,7,0.000000
1550,CN_W_to_MT_CN_W,7,0.000000
1551,CN_W_to_ID_CN_W,7,0.000000
1552,CN_W_to_WA_CN_W,7,0.000000


In [4]:
# redisplay qstep production

df_qstep = df_qstep_production.copy()

df_qstep = df_qstep.pivot_table(index=['producer', 'day'], columns='qstep', values='step_prod').reset_index()

df_qstep['total production'] = df_qstep[list(range(1,21))].sum(axis=1)

df_qstep = df_qstep.sort_values(by=['day','producer'])

df_qstep.round(3).to_csv(os.path.join(output_review_filepath, 'node_qstep_production_all.csv'), index=False)

df_qstep

qstep,producer,day,1,2,3,4,5,6,7,8,...,12,13,14,15,16,17,18,19,20,total production
0,AL_FD_qps,1,0.000000,0.078368,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.078368
7,AL_N_qps,1,0.030907,0.123629,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.154536
14,AL_ST_qps,1,0.000000,0.175218,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.175218
21,AL_S_qps,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
28,AR_qps,1,0.384323,0.896754,0.039621,0.026681,0.040421,0.153139,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.540939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
370,TX_ST_qps,7,0.002195,0.008780,0.001085,0.000746,0.001153,0.002792,0.002792,0.002792,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022335
377,UT_qps,7,0.089611,0.358442,0.044313,0.030456,0.047054,0.113975,0.076070,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.759921
384,VA_qps,7,0.049126,0.196502,0.024293,0.016696,0.013088,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.299705
391,WV_qps,7,0.000000,4.853635,0.532160,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.385795


In [5]:
## map QPS to node and sum node Qstep production

df_node_qstep = df_qstep_production.copy()

df_node_qstep['node'] = df_node_qstep['producer'].str[:2]

df_node_qstep_total = df_node_qstep.groupby(['node', 'day']).agg({'step_prod': 'sum'}).reset_index()

df_node_qstep
df_node_qstep_total


,node,day,step_prod
0,AL,1,0.408122
1,AL,2,0.408287
2,AL,3,0.408453
3,AL,4,0.408619
4,AL,5,0.408785
...,...,...,...
219,WY,3,4.322875
220,WY,4,4.324643
221,WY,5,4.326411
222,WY,6,4.328179


In [6]:
# redisplay ftariff flows

df_ftariff = df_ftariff_step_flow.copy()

df_ftariff = df_ftariff.pivot_table(index=['line', 'day'], columns='ftariff_step', values='step_flow').reset_index()

df_ftariff['total flow'] = df_ftariff[list(range(1,6))].sum(axis=1)

df_ftariff = df_ftariff.merge(df_total_pipeline_flows, on=['line', 'day'], how='outer')

df_ftariff = df_ftariff.merge(df_pipeline_slack, on=['line', 'day'], how='outer')

# df_ftariff[['from','to']] = df_node_flows_total_pipeline['line'].str.split('_to_', expand=True)

df_ftariff.round(2).to_csv(os.path.join(output_review_filepath, 'ftariff_step_flow_all.csv'))

# df_ftariff.head(20)
df_ftariff

,line,day,1,2,3,4,5,total flow,total_pipeline_flow,pipeline_slack
0,AL_to_FL,1,2.872328,0.0,0.371268,0.0,0.0,3.243596,3.243596,0.0
1,AL_to_FL,2,2.872328,0.0,0.477833,0.0,0.0,3.350161,3.350161,0.0
2,AL_to_FL,3,2.872328,0.0,0.789714,0.0,0.0,3.662041,3.662041,0.0
3,AL_to_FL,4,2.872328,0.0,0.468600,0.0,0.0,3.340928,3.340928,0.0
4,AL_to_FL,5,2.872328,0.0,0.000000,0.0,0.0,2.872328,2.872328,0.0
...,...,...,...,...,...,...,...,...,...,...
1549,WY_to_UT,3,4.320193,0.0,0.000000,0.0,0.0,4.320193,4.320193,0.0
1550,WY_to_UT,4,4.561922,0.0,0.000000,0.0,0.0,4.561922,4.561922,0.0
1551,WY_to_UT,5,4.375600,0.0,0.000000,0.0,0.0,4.375600,4.375600,0.0
1552,WY_to_UT,6,3.756044,0.0,0.000000,0.0,0.0,3.756044,3.756044,0.0


In [7]:
## sum pipeline flows in and out for each node and add to node values
df_node_flows_total_pipeline = df_total_pipeline_flows.copy()

df_node_flows_total_pipeline[['from','to']] = df_node_flows_total_pipeline['line'].str.split('_to_', expand=True)

df_node_flows_total_pipeline_from = df_node_flows_total_pipeline.groupby(['from', 'day']).agg({'total_pipeline_flow': 'sum'}).reset_index()
df_node_flows_total_pipeline_from = df_node_flows_total_pipeline_from.rename(columns={'total_pipeline_flow': 'total_pipeline_flow_out'})

df_node_flows_total_pipeline_to = df_node_flows_total_pipeline.groupby(['to', 'day']).agg({'total_pipeline_flow': 'sum'}).reset_index()
df_node_flows_total_pipeline_to = df_node_flows_total_pipeline_to.rename(columns={'total_pipeline_flow': 'total_pipeline_flow_in'})

df_node_flows_ftariff = df_ftariff.copy()

df_node_flows_ftariff[['from','to']] = df_node_flows_ftariff['line'].str.split('_to_', expand=True)

df_node_flows_ftariff_from = df_node_flows_ftariff.groupby(['from', 'day']).agg({'total flow': 'sum'}).reset_index()
df_node_flows_ftariff_from = df_node_flows_ftariff_from.rename(columns={'total flow': 'total_ftariff_flow_out'})

df_node_flows_ftariff_to = df_node_flows_ftariff.groupby(['to', 'day']).agg({'total flow': 'sum'}).reset_index()
df_node_flows_ftariff_to = df_node_flows_ftariff_to.rename(columns={'total flow': 'total_ftariff_flow_in'})




df_node_flows_total_pipeline
# df_node_flows_ftariff
df_node_flows_total_pipeline_from
df_node_flows_total_pipeline_to
df_node_flows_ftariff_from
# df_node_flows_ftariff_to



,from,day,total_ftariff_flow_out
0,AL,1,6.727792
1,AL,2,6.834358
2,AL,3,7.146238
3,AL,4,6.825125
4,AL,5,6.194614
...,...,...,...
464,WY,3,5.434231
465,WY,4,5.831671
466,WY,5,5.675030
467,WY,6,4.923870


In [8]:
df_node_values = df_duals.copy()

dfs = [df_duals,df_node_storage_withdrawal, df_node_storage_injection, df_node_production_slack, df_node_storage_slack,
       df_daily_node_demand, df_daily_node_production]

# combine all node values into one dataframe using functools.reduce and pd.merge
df_node_values = reduce(lambda left, right: pd.merge(left, right, on=['node', 'day'], how='outer'), dfs)



df_node_values = df_node_values.merge(df_node_qstep_total, left_on=['node', 'day'], right_on=['node','day'], how='outer')

df_node_values = df_node_values.rename(columns={'step_prod': 'sum_of_Qstep_production'})

## add pipeline flows to node values
df_node_values = df_node_values.merge(df_node_flows_total_pipeline_from, left_on=['node', 'day'], right_on=['from', 'day'], how='outer').drop(columns='from')
df_node_values = df_node_values.merge(df_node_flows_total_pipeline_to, left_on=['node', 'day'], right_on=['to', 'day'], how='outer').drop(columns='to')
df_node_values = df_node_values.merge(df_node_flows_ftariff_from, left_on=['node', 'day'], right_on=['from', 'day'], how='outer').drop(columns='from')
df_node_values = df_node_values.merge(df_node_flows_ftariff_to, left_on=['node', 'day'], right_on=['to', 'day'], how='outer').drop(columns='to')


# find balance of production, flows and demand
# net_flow_balance is net flows out of the node. should be >= 0, if not, there is a problem
df_node_values['net_flow_balance'] = (df_node_values['sum_of_Qstep_production'] -  
                                    df_node_values['total_ftariff_flow_out'] + 
                                    df_node_values['total_ftariff_flow_in'] - 
                                    df_node_values['daily_node_demand'])

df_node_values.round(2).to_csv(os.path.join(output_review_filepath, 'node_values_all.csv'), index=False)


df_node_values

,node,day,node_dual_value,storage_withdrawal,storage_injection,production_slack,storage_slack,daily_node_demand,daily_node_production,sum_of_Qstep_production,total_pipeline_flow_out,total_pipeline_flow_in,total_ftariff_flow_out,total_ftariff_flow_in,net_flow_balance
0,AL,1,-0.18,0.0,0.000000,0.0,0.0,2.040668,0.408122,0.408122,6.727792,8.360338,6.727792,8.360338,4.440892e-16
1,AZ,1,-0.33,0.0,0.000000,0.0,0.0,1.148369,0.000031,0.000031,3.318472,4.751811,3.318472,4.751811,2.850000e-01
2,AR,1,-0.05,0.0,0.000000,0.0,0.0,0.877601,1.540939,1.540939,2.763530,2.100193,2.763530,2.100193,1.110223e-16
3,CA,1,-0.48,0.0,0.000000,0.0,0.0,6.801110,0.549696,0.549696,0.000000,6.586414,0.000000,6.586414,3.350000e-01
4,CO,1,0.00,0.0,1.888094,0.0,0.0,1.682291,5.407832,5.407832,1.837446,0.000000,1.837446,0.000000,1.888094e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,MX,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
500,MX,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
501,MX,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
502,MX,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN


In [9]:
# opens files to view, can be commented out if not needed
os.startfile(os.path.join(output_review_filepath, 'ftariff_step_flow_all.csv'))
os.startfile(os.path.join(output_review_filepath, 'node_qstep_production_all.csv'))
os.startfile(os.path.join(output_review_filepath, 'node_values_all.csv'))